# Question 2: Can we predict educational outcomes, such as average years of schooling and education attainment levels, based on the teacher-pupil ratio and other economic factors?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df_quality1 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/quality1.csv')
school15 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/school15.csv')
school25 = pd.read_csv('https://raw.githubusercontent.com/urbanobruno/data-science-project-stanford/master/prepared_data/school25.csv')

In [3]:
df_quality1.head()

,SHCODE,WBCTRY,COUNTRY,TEAPRI60,TEAPRI65,TEAPRI70,TEAPRI75,TEAPRI80,TEAPRI85,TEAPRI90,...,SHSALP75,SHSALP80,SHSALP85,SHSALP90,DROP70,DROP75,DROP80,DROP85,DROP90,CONTINENT
0,1,DZA,Algeria,38.299999,44.299999,40.000000,40.900002,35.200001,27.799999,27.700001,...,599.0,296.0,NaN,350.0,31.3,28.8,23.5,9.8,10.0,Africa
1,2,AGO,Angola,35.400002,45.200001,44.400002,NaN,31.500000,31.299999,31.900000,...,NaN,NaN,NaN,NaN,66.0,66.0,66.0,66.0,66.0,Africa
2,3,BEN,Benin,40.500000,41.700001,44.299999,48.299999,47.500000,33.000000,34.700001,...,411.0,769.0,NaN,NaN,64.0,64.0,64.0,64.0,60.0,Africa
3,4,BWA,Botswana,31.500000,40.000000,36.500000,33.099998,32.299999,32.000000,31.700001,...,262.0,252.0,287.0,NaN,50.4,18.2,26.7,10.7,20.0,Africa
4,5,HVO,Burkina Faso,NaN,NaN,NaN,47.099998,54.500000,56.500000,56.700001,...,1528.0,1184.0,884.0,NaN,51.0,38.3,25.1,26.2,29.0,Africa


In [4]:
school15.head()

,SHCODE,COUNTRY,YEAR,N015,PRI15,PRIC15,SEC15,SECC15,HIGH15,HIGHC15,TYR15,PYR15,SYR15,HYR15
0,1,Algeria,1960,80,17.1,4.9,2.6,0.5,0.3,0.2,0.94,0.83,0.1,0.01
1,1,Algeria,1965,81.9,13.7,4.4,3.9,1.2,0.4,0.2,0.98,0.8,0.17,0.01
2,1,Algeria,1970,73.7,19.2,6.4,6.7,2.3,0.3,0.0,1.47,1.19,0.27,0.01
3,1,Algeria,1975,66.5,24.3,8.3,8.4,2.5,0.8,0.2,1.89,1.53,0.35,0.02
4,1,Algeria,1980,57.6,29.1,10,11.7,3,1.6,0.4,2.51,1.97,0.5,0.04


In [5]:
school25.head()

,SHCODE,COUNTRY,YEAR,N025,PRI25,PRIC25,SEC25,SECC25,HIGH25,HIGHC25,TYR25,PYR25,SYR25,HYR25
0,1,Algeria,1960,80.4,17.1,4.9,2.2,0.8,0.3,0.2,0.94,0.81,0.11,0.01
1,1,Algeria,1965,88,9.8,3.2,1.7,0.7,0.4,0.3,0.63,0.51,0.1,0.01
2,1,Algeria,1970,84.4,13,4.4,2.2,1.1,0.3,0.2,0.8,0.67,0.12,0.01
3,1,Algeria,1975,80.2,16.6,5.6,2.6,1.3,0.6,0.4,1.04,0.86,0.16,0.02
4,1,Algeria,1980,72.5,22.8,7.8,3.5,1.8,1.2,0.8,1.48,1.2,0.24,0.04


In [6]:
df_quality1 = df_quality1[[
    'SHCODE', 'COUNTRY', 'TEAPRI60', 'TEAPRI65', 'TEAPRI70', 'TEAPRI75', 'TEAPRI80', 'TEAPRI85', 'TEAPRI90',
    'GEEPRI60','GEEPRI65','GEEPRI70','GEEPRI75','GEEPRI80','GEEPRI85','GEEPRI90',
    'SHPUPP60','SHPUPP65','SHPUPP70','SHPUPP75','SHPUPP80','SHPUPP85','SHPUPP90',
    ]]
df_combined = school25[['SHCODE', 'COUNTRY', 'YEAR', 'TYR25']]

In [7]:
df_quality1

,SHCODE,COUNTRY,TEAPRI60,TEAPRI65,TEAPRI70,TEAPRI75,TEAPRI80,TEAPRI85,TEAPRI90,GEEPRI60,...,GEEPRI80,GEEPRI85,GEEPRI90,SHPUPP60,SHPUPP65,SHPUPP70,SHPUPP75,SHPUPP80,SHPUPP85,SHPUPP90
0,1,Algeria,38.299999,44.299999,40.000000,40.900002,35.200001,27.799999,27.700001,NaN,...,239.0,NaN,370.0,NaN,15.500000,17.799999,15.500000,8.600000,NaN,13.300000
1,2,Angola,35.400002,45.200001,44.400002,NaN,31.500000,31.299999,31.900000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Benin,40.500000,41.700001,44.299999,48.299999,47.500000,33.000000,34.700001,NaN,...,202.0,NaN,NaN,NaN,24.700001,25.700001,10.500000,18.200001,NaN,NaN
3,4,Botswana,31.500000,40.000000,36.500000,33.099998,32.299999,32.000000,31.700001,78.0,...,211.0,207.0,207.0,14.5,12.000000,11.800000,9.500000,11.200000,9.200000,NaN
4,5,Burkina Faso,NaN,NaN,NaN,47.099998,54.500000,56.500000,56.700001,NaN,...,104.0,84.0,NaN,NaN,NaN,NaN,33.200001,22.000000,16.299999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,137,Vanuatu,31.799999,29.200001,23.500000,33.500000,23.600000,24.100000,19.600000,NaN,...,NaN,NaN,273.0,NaN,NaN,NaN,NaN,NaN,NaN,16.299999
119,156,Albania,NaN,26.800000,26.200000,NaN,21.300000,20.000000,19.100000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,150,Bulgaria,NaN,22.600000,22.000000,20.200000,19.300000,17.700000,15.400000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,155,Cuba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_combined.head()

,SHCODE,COUNTRY,YEAR,TYR25
0,1,Algeria,1960,0.94
1,1,Algeria,1965,0.63
2,1,Algeria,1970,0.8
3,1,Algeria,1975,1.04
4,1,Algeria,1980,1.48


In [12]:
df_quality_filtered = df_quality1[df_quality1['COUNTRY'].isin(df_combined['COUNTRY'])]
df_quality_filtered

,SHCODE,COUNTRY,TEAPRI60,TEAPRI65,TEAPRI70,TEAPRI75,TEAPRI80,TEAPRI85,TEAPRI90,GEEPRI60,...,GEEPRI80,GEEPRI85,GEEPRI90,SHPUPP60,SHPUPP65,SHPUPP70,SHPUPP75,SHPUPP80,SHPUPP85,SHPUPP90
0,1,Algeria,38.299999,44.299999,40.000000,40.900002,35.200001,27.799999,27.700001,NaN,...,239.0,NaN,370.0,NaN,15.500000,17.799999,15.5,8.600000,NaN,13.3
2,3,Benin,40.500000,41.700001,44.299999,48.299999,47.500000,33.000000,34.700001,NaN,...,202.0,NaN,NaN,NaN,24.700001,25.700001,10.5,18.200001,NaN,NaN
3,4,Botswana,31.500000,40.000000,36.500000,33.099998,32.299999,32.000000,31.700001,78.0,...,211.0,207.0,207.0,14.5,12.000000,11.800000,9.5,11.200000,9.20,NaN
6,7,Cameroon,46.700001,47.200001,47.700001,50.599998,51.500000,50.799999,51.099998,NaN,...,75.0,95.0,79.0,NaN,6.900000,7.600000,NaN,5.800000,6.60,6.4
9,12,Congo,52.700001,59.900002,61.900002,58.700001,54.400002,61.400002,65.900002,136.0,...,162.0,133.0,NaN,12.8,11.600000,10.700000,9.7,8.900000,5.20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,133,New Zealand,30.900000,25.200001,21.299999,18.500000,16.700001,19.900000,18.000000,407.0,...,1680.0,1730.0,1894.0,5.1,8.200000,11.000000,13.0,16.400000,15.30,16.5
115,134,Papua New Guinea,35.099998,31.000000,29.700001,31.600000,31.400000,30.799999,31.700001,NaN,...,NaN,NaN,NaN,NaN,14.300000,9.400000,24.0,NaN,NaN,NaN
120,150,Bulgaria,NaN,22.600000,22.000000,20.200000,19.300000,17.700000,15.400000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,155,Cuba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# KNN finds best parameters:

def findsBestParametersKNN(pipeline, X, y, rangeK=range(1, 40)):
    grid_cv = GridSearchCV(
        pipeline,
        {
            'kneighborsregressor__n_neighbors': rangeK,
            'kneighborsregressor__metric': ['euclidean', 'manhattan']
        },
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        error_score='raise'
        )

    grid_cv.fit(X, y)

    return grid_cv.best_estimator_

In [14]:
def calculatesRMSEusingCV(model, X, y):
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    return cv_rmse

In [ ]:
df_quality_filtered

,SHCODE,COUNTRY,CONTINENT,TEAPRI60,TEAPRI65,TEAPRI70,TEAPRI75,TEAPRI80,TEAPRI85,TEAPRI90
0,1,Algeria,Africa,38.299999,44.299999,40.000000,40.900002,35.200001,27.799999,27.700001
2,3,Benin,Africa,40.500000,41.700001,44.299999,48.299999,47.500000,33.000000,34.700001
3,4,Botswana,Africa,31.500000,40.000000,36.500000,33.099998,32.299999,32.000000,31.700001
6,7,Cameroon,Africa,46.700001,47.200001,47.700001,50.599998,51.500000,50.799999,51.099998
9,12,Congo,Africa,52.700001,59.900002,61.900002,58.700001,54.400002,61.400002,65.900002
...,...,...,...,...,...,...,...,...,...,...
114,133,New Zealand,Oceania,30.900000,25.200001,21.299999,18.500000,16.700001,19.900000,18.000000
115,134,Papua New Guinea,Oceania,35.099998,31.000000,29.700001,31.600000,31.400000,30.799999,31.700001
120,150,Bulgaria,Europe,NaN,22.600000,22.000000,20.200000,19.300000,17.700000,15.400000
121,155,Cuba,Americas,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
teacher_features = ['TEAPRI60', 'TEAPRI65', 'TEAPRI70', 'TEAPRI75', 'TEAPRI80', 'TEAPRI85', 'TEAPRI90']
gee_features = ['GEEPRI60', 'GEEPRI65', 'GEEPRI70', 'GEEPRI75', 'GEEPRI80', 'GEEPRI85', 'GEEPRI90']
gdp_features = ['SHPUPP60', 'SHPUPP65', 'SHPUPP70', 'SHPUPP75', 'SHPUPP80', 'SHPUPP85', 'SHPUPP90']

# Function to map the correct TEAPRI column based on the YEAR
def get_teapri(row, list):

    if row['COUNTRY'] not in df_combined['COUNTRY'].values:
        return np.nan

    df_temp = df_quality_filtered[df_quality_filtered['COUNTRY'] == row['COUNTRY']]

    if df_temp.empty:
      return np.nan

    if row['YEAR'] == 1960:
        return df_temp[list[0]].values[0]
    elif row['YEAR'] == 1965:
        return df_temp[list[1]].values[0]
    elif row['YEAR'] == 1970:
        return df_temp[list[2]].values[0]
    elif row['YEAR'] == 1975:
        return df_temp[list[3]].values[0]
    elif row['YEAR'] == 1980:
        return df_temp[list[4]].values[0]
    elif row['YEAR'] == 1985:
        return df_temp[list[5]].values[0]
    elif row['YEAR'] == 1990:
        return df_temp[list[6]].values[0]
    else:
        return np.nan

df_combined['TEAPRI'] = df_combined.apply(get_teapri, args=(teacher_features,), axis=1)
df_combined['GEEPRI'] = df_combined.apply(get_teapri, args=(gee_features,), axis=1)
df_combined['SHPUPP'] = df_combined.apply(get_teapri, args=(gdp_features,), axis=1)


In [21]:
df_combined = df_combined.dropna()
df_combined

,SHCODE,COUNTRY,YEAR,TYR25,TEAPRI,GEEPRI,SHPUPP
1,1,Algeria,1965,0.63,44.299999,237.0,15.500000
2,1,Algeria,1970,0.8,40.000000,326.0,17.799999
3,1,Algeria,1975,1.04,40.900002,358.0,15.500000
4,1,Algeria,1980,1.48,35.200001,239.0,8.600000
6,1,Algeria,1990,2.81,27.700001,370.0,13.300000
...,...,...,...,...,...,...,...
730,134,Papua New Guinea,1965,1.01,31.000000,219.0,14.300000
731,134,Papua New Guinea,1970,0.58,29.700001,164.0,9.400000
732,134,Papua New Guinea,1975,0.92,31.600000,415.0,24.000000
775,153,Romania,1980,7.09,20.600000,165.0,11.610000


In [24]:
df_model = df_combined.copy()
df_model

,SHCODE,COUNTRY,YEAR,TYR25,TEAPRI,GEEPRI,SHPUPP
1,1,Algeria,1965,0.63,44.299999,237.0,15.500000
2,1,Algeria,1970,0.8,40.000000,326.0,17.799999
3,1,Algeria,1975,1.04,40.900002,358.0,15.500000
4,1,Algeria,1980,1.48,35.200001,239.0,8.600000
6,1,Algeria,1990,2.81,27.700001,370.0,13.300000
...,...,...,...,...,...,...,...
730,134,Papua New Guinea,1965,1.01,31.000000,219.0,14.300000
731,134,Papua New Guinea,1970,0.58,29.700001,164.0,9.400000
732,134,Papua New Guinea,1975,0.92,31.600000,415.0,24.000000
775,153,Romania,1980,7.09,20.600000,165.0,11.610000


In [25]:
df_model['TEAPRI'] = pd.to_numeric(df_model['TEAPRI'], errors='coerce')
df_model['TYR25'] = pd.to_numeric(df_model['TYR25'], errors='coerce')
df_model['GEEPRI'] = pd.to_numeric(df_model['GEEPRI'], errors='coerce')
df_model['SHPUPP'] = pd.to_numeric(df_model['SHPUPP'], errors='coerce')

df_model.replace([np.inf, -np.inf], np.nan, inplace=True)
df_model.dropna(inplace=True)

# Correlations with all features: Teacher-Pupil Ratio, Real Government Educational Expenditure Per Pupil, Ratio of GEEPRI to Real Per Capita GDP

In [26]:
y = df_model['TYR25']
X = df_model[['TEAPRI','GEEPRI','SHPUPP']]

In [30]:
numeric_features = ['TEAPRI','GEEPRI','SHPUPP']

knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)

findsBestParametersKNN(knn_pipeline, X, y)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor',
                 KNeighborsRegressor(metric='manhattan', n_neighbors=13))])

In [31]:
best_knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(metric='manhattan', n_neighbors=13)
)

linear_pipeline = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

rmse_allfeatures_knn = calculatesRMSEusingCV(best_knn_pipeline, X, y)
rmse_allfeatures_linear = calculatesRMSEusingCV(linear_pipeline, X, y)

rmse_allfeatures_knn, rmse_allfeatures_linear

(1.6149266513727603, 1.8434957309129296)

# Correlations with all features: Teacher-Pupil Ratio + Economic Factors

In [34]:
features = ['TEAPRI','GEEPRI','SHPUPP']

dic = dict()

for feature in features:
    dic[feature] = df_model[feature].corr(df_model['TYR25'])

corrs = pd.Series(dic, name='Correlation')
corrs.sort_values()


,Correlation
TEAPRI,-0.675885
SHPUPP,0.205242
GEEPRI,0.676471


# Model Teacher-Pupil Ratio + Government Expenditure Features


In [45]:
y = df_model['TYR25']
X = df_model[['TEAPRI','GEEPRI']]

knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)

findsBestParametersKNN(knn_pipeline, X, y)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor',
                 KNeighborsRegressor(metric='manhattan', n_neighbors=36))])

In [46]:
best_knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(metric='manhattan', n_neighbors=36)
)

linear_pipeline = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

rmse_without_c_knn = calculatesRMSEusingCV(best_knn_pipeline, X, y)
rmse_without_c_linear = calculatesRMSEusingCV(linear_pipeline, X, y)

rmse_without_c_knn, rmse_without_c_linear


(1.7130431932782617, 1.9835406429559703)

# Model Government Education Expenditure Feature



In [43]:
y = df_model['TYR25']
X = df_model[['GEEPRI']]

knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)

findsBestParametersKNN(knn_pipeline, X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsregressor',
                 KNeighborsRegressor(metric='euclidean', n_neighbors=33))])

In [44]:
best_knn_pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(metric='euclidean', n_neighbors=33)
)

linear_pipeline = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

rmse_without_y_knn = calculatesRMSEusingCV(best_knn_pipeline, X, y)
rmse_without_y_linear = calculatesRMSEusingCV(linear_pipeline, X, y)

rmse_without_y_knn, rmse_without_y_linear


(1.8111531693015257, 2.337427866144101)

# Evaluation

In [51]:
data = {
    'Model': ['KNN', 'Linear Regression'],
    'All Features': [rmse_allfeatures_knn, rmse_allfeatures_linear],
    'TEAPRI & GEEPRI': [rmse_without_c_knn, rmse_without_c_linear],
    'GEEPRI': [rmse_without_y_knn, rmse_without_y_linear],
}
df_rmse = pd.DataFrame(data)

df_rmse

,Model,All Features,TEAPRI & GEEPRI,GEEPRI
0,KNN,1.614927,1.713043,1.811153
1,Linear Regression,1.843496,1.983541,2.337428


# Conclusion

We sought to analyze whether the economic factors and teacher pupil ratio could help in predicting the average years of schooling. Based on our analysis, it appears that when comparing the 3 features: teacher-pupil ratio, government expenditure, and ratio of government expenditure per capital GDP, that the most accurate prediction when including all 3 features. This suggests that when we had more data, it was better at predicting, as indicated by the lowest value of 1.61 KNN analysis